# Execution of the Bell circuit

The usual first experiment in Quantum Computing is to simulate a 2-qubit Bell state.

## Creating the EPR/Bell state 

Let us create first the related circuit.

In [1]:
from mpqp import QCircuit
from mpqp.gates import H, CNOT

In [2]:
circuit = QCircuit([H(0), CNOT(0,1)], label="Bell state")
print(circuit)

     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘


## Run the circuit on a local device

In [3]:
from mpqp.execution import run, ATOSDevice

In [4]:
result = run(circuit, ATOSDevice.MYQLM_PYLINALG)
print(result)
print(result.amplitudes)
print(result.probabilities)

Result: ATOSDevice, MYQLM_PYLINALG

        State vector: [0.70710677+0.j 0.        +0.j 0.        +0.j 0.70710677+0.j]
        Probabilities: [0.5 0.  0.  0.5]
        Number of qubits: 2


[0.70710677+0.j 0.        +0.j 0.        +0.j 0.70710677+0.j]
[0.5 0.  0.  0.5]


In [5]:
from mpqp.measures import BasisMeasure

In [6]:
circuit.add(BasisMeasure([0,1], shots=0))

In [7]:
result = run(circuit, ATOSDevice.MYQLM_PYLINALG)
print(result)

Result: ATOSDevice, MYQLM_PYLINALG

        State vector: [0.70710677+0.j 0.        +0.j 0.        +0.j 0.70710677+0.j]
        Probabilities: [0.5 0.  0.  0.5]
        Number of qubits: 2




In [8]:
circuit = circuit.without_measurements()
circuit.add(BasisMeasure([0,1], shots=1024))

In [9]:
result = run(circuit, ATOSDevice.MYQLM_PYLINALG)
print(result)
print(result.samples)
print(result.counts)
print(result.probabilities)

Result: ATOSDevice, MYQLM_PYLINALG
Counts: [516, 0, 0, 508]
Probabilities: [0.50390625 0.         0.         0.49609375]
State: 11, Index: 3, Count: 508, Probability: 0.49609375
State: 00, Index: 0, Count: 516, Probability: 0.50390625
Error: 0.01563215790957881


[State: 11, Index: 3, Count: 508, Probability: 0.49609375, State: 00, Index: 0, Count: 516, Probability: 0.50390625]
[516, 0, 0, 508]
[0.50390625 0.         0.         0.49609375]


## Run the circuit on multiple devices

In [10]:
from mpqp.execution import IBMDevice, AWSDevice

In [12]:
results = run(circuit, [ATOSDevice.MYQLM_PYLINALG, IBMDevice.AER_SIMULATOR, AWSDevice.BRAKET_LOCAL_SIMULATOR])
print(results)
print('---------')
print(results[0])

This program uses OpenQASM language features that may not be supported on QPUs or on-demand simulators.


BatchResult: 3 results
Result: AWSDevice, BRAKET_LOCAL_SIMULATOR
Counts: [507, 0, 0, 517]
Probabilities: [0.49511719 0.         0.         0.50488281]
State: 00, Index: 0, Count: 507, Probability: 0.4951171875
State: 11, Index: 3, Count: 517, Probability: 0.5048828125
Error: None

Result: IBMDevice, AER_SIMULATOR
Counts: [492, 0, 0, 532]
Probabilities: [0.48046875 0.         0.         0.51953125]
State: 00, Index: 0, Count: 492, Probability: 0.48046875
State: 11, Index: 3, Count: 532, Probability: 0.51953125
Error: None

Result: ATOSDevice, MYQLM_PYLINALG
Counts: [510, 0, 0, 514]
Probabilities: [0.49804688 0.         0.         0.50195312]
State: 00, Index: 0, Count: 510, Probability: 0.498046875
State: 11, Index: 3, Count: 514, Probability: 0.501953125
Error: 0.01563251571902321


---------
Result: AWSDevice, BRAKET_LOCAL_SIMULATOR
Counts: [507, 0, 0, 517]
Probabilities: [0.49511719 0.         0.         0.50488281]
State: 00, Index: 0, Count: 507, Probability: 0.4951171875
State: 11

## Run or submit the circuit on a remote device

In [26]:
result = run(circuit, ATOSDevice.QLM_LINALG)
print(result)

Logging as user hjaffali...
Submitted a new batch: Job9831
Result: ATOSDevice, QLM_LINALG
Counts: [500, 0, 0, 524]
Probabilities: [0.48828125 0.         0.         0.51171875]
State: 00, Index: 0, Count: 500, Probability: 0.48828125
State: 11, Index: 3, Count: 524, Probability: 0.51171875
Error: 0.015628340765779864




In [28]:
from mpqp.execution import submit

In [30]:
job_id, job = submit(circuit, IBMDevice.IBMQ_QASM_SIMULATOR)
print(job_id)

cn8nvgv79cpiv1f53brg


In [31]:
from mpqp.execution import remote_result_from_id

In [32]:
result = remote_result_from_id(job_id, IBMDevice.IBMQ_QASM_SIMULATOR)
print(result)

Result: IBMDevice, IBMQ_QASM_SIMULATOR
Counts: [509, 0, 0, 515]
Probabilities: [0.49707031 0.         0.         0.50292969]
State: 00, Index: 0, Count: 509, Probability: 0.4970703125
State: 11, Index: 3, Count: 515, Probability: 0.5029296875
Error: None


